In [1]:
import os
import torch
import GPUtil
from sklearn.model_selection import train_test_split

# Imports de la libreria propia
from vecopsciml.kernels.derivative import DerivativeKernels
from vecopsciml.utils import TensOps

# Imports de las funciones creadas para este programa
from model.baseline_model import BaselineNonlinearModel
from utils.folders import create_folder
from utils.load_data import load_data
from trainers.train import train_loop

In [2]:
import matplotlib.pyplot as plt
from vecopsciml.operators.zero_order import Mx, My

In [ ]:
# Creamos los paths para las distintas carpetas
ROOT_PATH = r'/home/rmunoz/Escritorio/rmunozTMELab/Physically-Guided-Machine-Learning'
DATA_PATH = os.path.join(ROOT_PATH, r'data/non_linear_100_0/non_linear_100_0.pkl')
RESULTS_FOLDER_PATH = os.path.join(ROOT_PATH, r'transfer_learning/results/non_linear')
MODEL_RESULTS_PATH = os.path.join(ROOT_PATH, r'transfer_learning/results/non_linear/baseline')

# Creamos las carpetas que sean necesarias (si ya están creadas se avisará de ello)
create_folder(RESULTS_FOLDER_PATH)
create_folder(MODEL_RESULTS_PATH)

Folder already exists at: /home/rmunoz/Escritorio/rmunozTMELab/Physically-Guided-Machine-Learning/transfer_learning/results/non_linear
Folder already exists at: /home/rmunoz/Escritorio/rmunozTMELab/Physically-Guided-Machine-Learning/transfer_learning/results/non_linear/baseline


In [8]:
# Load dataset
dataset = load_data(DATA_PATH)

Data successfully loaded from: /home/rmunoz/Escritorio/rmunozTMELab/Physically-Guided-Machine-Learning/data/non_linear_100_0/non_linear_100_0.pkl


In [9]:
# Convolutional filters to derivate
dx = dataset['x_step_size']
dy = dataset['y_step_size']
D = DerivativeKernels(dx, dy, 0).grad_kernels_two_dimensions()

In [10]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Using device: {DEVICE}")

Using device: cuda


In [11]:
# Train data splitting in train/test
X = torch.tensor(dataset['X_train'], dtype=torch.float32).unsqueeze(1)
y = torch.tensor(dataset['y_train'], dtype=torch.float32).unsqueeze(1)
K = torch.tensor(dataset['k_train'], dtype=torch.float32).unsqueeze(1)
f = torch.tensor(dataset['f_train'], dtype=torch.float32).unsqueeze(1)

X_train, X_test, y_train, y_test, K_train, K_test, f_train, f_test = train_test_split(X, y, K, f, test_size=0.3, random_state=42)

# Data processing and adequacy with our TensOps library
X_train = X_train.to(DEVICE)
X_test = X_test.to(DEVICE)

y_train = TensOps(y_train.to(DEVICE).requires_grad_(True), space_dimension=2, contravariance=0, covariance=0)
y_test = TensOps(y_test.to(DEVICE).requires_grad_(True), space_dimension=2, contravariance=0, covariance=0)

K_train = TensOps(K_train.to(DEVICE).requires_grad_(True), space_dimension=2, contravariance=0, covariance=0)
K_test = TensOps(K_test.to(DEVICE).requires_grad_(True), space_dimension=2, contravariance=0, covariance=0)

f_train = TensOps(f_train.to(DEVICE).requires_grad_(True), space_dimension=2, contravariance=0, covariance=0)
f_test = TensOps(f_test.to(DEVICE).requires_grad_(True), space_dimension=2, contravariance=0, covariance=0)

# Loading and processing validation data
X_val = torch.tensor(dataset['X_val'], dtype=torch.float32).unsqueeze(1)
y_val = TensOps(torch.tensor(dataset['y_val'], dtype=torch.float32, requires_grad=True).unsqueeze(1), space_dimension=2, contravariance=0, covariance=0)
K_val = TensOps(torch.tensor(dataset['k_val'], dtype=torch.float32, requires_grad=True).unsqueeze(1), space_dimension=2, contravariance=0, covariance=0)
f_val = TensOps(torch.tensor(dataset['f_val'], dtype=torch.float32, requires_grad=True).unsqueeze(1), space_dimension=2, contravariance=0, covariance=0)

In [12]:
n_modes = 10

# Predictive network architecture
input_shape = X_train[0].shape
predictive_layers = [20, 10, n_modes, 10, 20]
predictive_output = y_train.values[0].shape

# Explanatory network architecture
explanatory_input = Mx(My(y_train)).values[0].shape
explanatory_layers = [10, 10]
explanatory_output = Mx(My(f_train)).values[0].shape

# Other parameters
n_filters_explanatory = 5

In [13]:
# Load model and the optimizer
model = BaselineNonlinearModel(input_shape, predictive_layers, predictive_output, explanatory_input, explanatory_layers, explanatory_output, n_filters_explanatory).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=3e-3)

# Parametros de entrenamiento
start_epoch = 0
n_epochs = 20000

batch_size = 64
n_checkpoints = 10

train_loop(model, optimizer, X_train, y_train, f_train, X_test, y_test, f_test,
           D,  n_checkpoints, start_epoch=start_epoch, n_epochs=n_epochs, batch_size=batch_size, 
           model_results_path=MODEL_RESULTS_PATH, device=DEVICE)

Starting training from scratch.
Epoch 0, Train loss: 1.348e+09, Test loss: 1.459e+09, MSE(e): 9.300e+01, MSE(pi1): 4.113e+04, MSE(pi2): 3.929e+01, MSE(pi3): 6.284e+01
Epoch 100, Train loss: 2.997e+07, Test loss: 3.594e+07, MSE(e): 2.990e+00, MSE(pi1): 3.821e+00, MSE(pi2): 1.378e+00, MSE(pi3): 2.929e-01
Epoch 200, Train loss: 1.994e+07, Test loss: 2.314e+07, MSE(e): 1.982e+00, MSE(pi1): 9.573e+00, MSE(pi2): 9.543e-01, MSE(pi3): 2.768e-01
Epoch 300, Train loss: 7.217e+06, Test loss: 8.507e+06, MSE(e): 7.094e-01, MSE(pi1): 9.458e+00, MSE(pi2): 4.449e-01, MSE(pi3): 2.746e-01
Epoch 400, Train loss: 5.814e+06, Test loss: 6.681e+06, MSE(e): 5.728e-01, MSE(pi1): 5.957e+00, MSE(pi2): 3.840e-01, MSE(pi3): 2.600e-01
Epoch 500, Train loss: 5.294e+06, Test loss: 6.027e+06, MSE(e): 5.224e-01, MSE(pi1): 4.379e+00, MSE(pi2): 3.598e-01, MSE(pi3): 2.501e-01
Epoch 600, Train loss: 5.011e+06, Test loss: 5.629e+06, MSE(e): 4.950e-01, MSE(pi1): 3.630e+00, MSE(pi2): 3.461e-01, MSE(pi3): 2.434e-01
Epoch 700, 

In [14]:
# Parametros de entrenamiento
start_epoch = 18000
n_epochs = 100000

batch_size = 64
n_checkpoints = 5

second_lr = 3e-4

train_loop(model, optimizer, X_train, y_train, f_train, X_test, y_test, f_test,
           D,  n_checkpoints, start_epoch=start_epoch, n_epochs=n_epochs, batch_size=batch_size, 
           model_results_path=MODEL_RESULTS_PATH, device=DEVICE, new_lr=second_lr)

Starting training from a checkpoint. Epoch 18000.
Epoch 18000, Train loss: 3.640e+03, Test loss: 2.332e+05, MSE(e): 2.989e-04, MSE(pi1): 4.944e-02, MSE(pi2): 2.503e-04, MSE(pi3): 1.570e-03
Epoch 18100, Train loss: 3.271e+03, Test loss: 2.336e+05, MSE(e): 2.888e-04, MSE(pi1): 2.300e-02, MSE(pi2): 2.466e-04, MSE(pi3): 1.533e-03
Epoch 18200, Train loss: 3.265e+03, Test loss: 2.335e+05, MSE(e): 2.883e-04, MSE(pi1): 2.301e-02, MSE(pi2): 2.462e-04, MSE(pi3): 1.519e-03
Epoch 18300, Train loss: 3.259e+03, Test loss: 2.333e+05, MSE(e): 2.878e-04, MSE(pi1): 2.299e-02, MSE(pi2): 2.458e-04, MSE(pi3): 1.513e-03
Epoch 18400, Train loss: 3.253e+03, Test loss: 2.332e+05, MSE(e): 2.873e-04, MSE(pi1): 2.294e-02, MSE(pi2): 2.454e-04, MSE(pi3): 1.509e-03
Epoch 18500, Train loss: 3.247e+03, Test loss: 2.331e+05, MSE(e): 2.868e-04, MSE(pi1): 2.288e-02, MSE(pi2): 2.450e-04, MSE(pi3): 1.507e-03
Epoch 18600, Train loss: 3.241e+03, Test loss: 2.330e+05, MSE(e): 2.862e-04, MSE(pi1): 2.281e-02, MSE(pi2): 2.446e-0